In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# ---------------- Synthetic dataset ----------------
np.random.seed()
N = 100000

route_planning = pd.DataFrame({
    'origin_x': np.random.uniform(0, 100, N),
    'origin_y': np.random.uniform(0, 100, N),
    'dest_x': np.random.uniform(0, 100, N),
    'dest_y': np.random.uniform(0, 100, N),
    'traffic_density': np.random.uniform(0, 1, N),
    'road_type': np.random.choice([1, 2, 3], N),
})
route_planning['distance'] = np.sqrt((route_planning['dest_x'] - route_planning['origin_x'])**2 +
                                     (route_planning['dest_y'] - route_planning['origin_y'])**2)
speed_base = {1:50, 2:40, 3:30}
route_planning['speed'] = route_planning['road_type'].map(speed_base) * np.random.uniform(0.8,1.2,N)
route_planning['travel_time'] = (route_planning['distance']/route_planning['speed'])*60*\
                                (1+route_planning['traffic_density']*np.random.uniform(0.1,0.5,N))

vehicle_assignment = pd.DataFrame({
    'vehicle_capacity': np.random.randint(50,200,N),
    'battery_level': np.random.uniform(0.3,1.0,N),

    'delivery_size': np.random.randint(5,50,N),
    'vehicle_type': np.random.choice([1,2],N),
    'speed_factor': np.random.uniform(0.9,1.1,N),
})
vehicle_assignment['assigned_speed'] = route_planning['speed']*vehicle_assignment['speed_factor']
vehicle_assignment['load_utilization'] = vehicle_assignment['delivery_size']/vehicle_assignment['vehicle_capacity']

time_scheduling = pd.DataFrame({
    'requested_time': np.random.randint(8,20,N),
    'delivery_priority': np.random.randint(1,5,N),
    'customer_patience': np.random.uniform(0,1,N),
})
time_scheduling['delay_probability'] = np.clip(
    (route_planning['travel_time']/60)*(1+vehicle_assignment['load_utilization']*0.5)*np.random.uniform(0.8,1.2,N),
    0,1
)

dynamic_rerouting = pd.DataFrame({
    'current_x': np.random.uniform(0,100,N),
    'current_y': np.random.uniform(0,100,N),
    'traffic_updates': np.random.uniform(0,1,N),
    'new_delivery_requests': np.random.randint(0,3,N),
    'vehicle_status': np.random.choice([0,1],N),
    'weather': np.random.choice([0,1],N),
})
dynamic_rerouting['congestion_score'] = dynamic_rerouting['traffic_updates'] + \
                                       dynamic_rerouting['new_delivery_requests']*0.5 + \
                                       dynamic_rerouting['weather']*0.5 + \
                                       (route_planning['travel_time']/route_planning['travel_time'].max())*0.5

# ---------------- Combine and normalize ----------------
datasets = [route_planning, vehicle_assignment, time_scheduling, dynamic_rerouting]
dataset_dims = [df.shape[1] for df in datasets]
max_dim = max(dataset_dims)

padded_data = []
for df in datasets:
    arr = df.values
    if arr.shape[1] < max_dim:
        arr = np.hstack([arr, np.zeros((arr.shape[0], max_dim - arr.shape[1]))])
    padded_data.append(arr)

DATA_MATRIX = np.hstack(padded_data)
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0)+1e-8)

def generate_targets(DATA_MATRIX, candidate_dims, D_graph):
    targets = []
    for node_idx in range(D_graph):
        row = DATA_MATRIX[node_idx % len(DATA_MATRIX)]
        node_targets = {}
        for dim in candidate_dims:
            if len(row) >= dim:
                sampled = row[:dim]
            else:
                sampled = np.pad(row, (0, dim - len(row)), constant_values=0.5)
            node_targets[dim] = sampled
        targets.append(node_targets)
    return targets
candidate_dims = [4,4,4,4]#[6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,]
D_graph = 4
synthetic_targets = generate_targets(DATA_MATRIX, candidate_dims, D_graph)

# ---------------- Targets ----------------
#candidate_dims = [6,6,6,6,6,6,6]#,6,6,6,6,6,6,6,6,6,6,6,6,]
#D_graph = 4
inner_archive_size = 120
inner_offspring = 80
outer_archive_size = 80
outer_offspring = 80
inner_iters_per_outer = 15
outer_generations = 500
outer_cost_limit = 1350
inner_learning = 0.9


In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# ---------------- CONFIG ----------------
candidate_dims = [4] *10
D_graph = 4

inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_iters_per_outer = 50
outer_generations = 100
outer_cost_limit = 10000
inner_learning = 0.1
gamma_interlayer = 1
top_k = 2630
np.random.seed()
class MetricsEvaluator:
    def __init__(self, data_matrix):
        self.data_matrix = data_matrix
        self.num_features = data_matrix.shape[1]
        self.W0, self.T0, self.U0 = 15.0, 100.0, 10.0
        self.P0 = 10.0  # baseline for patience

    def compute_node_metrics(self, node_idx, y=None):
        row = self.data_matrix[node_idx % self.data_matrix.shape[0]]
        k = self.num_features
        base = max(1, k // 3)
        wait_cols = slice(0, base)
        thr_cols = slice(base, 2*base)
        util_cols = slice(2*base, k)

        wait_signal = np.mean(row[wait_cols])
        throughput_signal = np.mean(row[thr_cols])
        util_signal = np.mean(row[util_cols])

        if y is None:
            y = np.array([0.5, 0.5, 0.5])
        else:
            y = np.array(y[:3]) if len(y) >= 3 else np.pad(y, (0, 3-len(y)), constant_values=0.5)

        # Original metrics
        wait = np.clip(self.W0*(1 + 1.2*wait_signal + 1*y[0]), 0, 150)
        throughput = np.clip(self.T0*(1 + 1.1*throughput_signal + 0.6*y[1] - 0.4*wait_signal), 0, 150)
        util = np.clip(self.U0 + 1.2*util_signal + 1.5*y[2], 0, 150)

        # --- New metric: patience ---
        patience_signal = np.mean(row)  # simple average as proxy
        patience = np.clip(self.P0 * (1 + 0.5*patience_signal + 0.5*y[0]), 0, 200)  # scaled like others

        # Combine into score
        score = -wait + throughput + util + patience  # add patience like others

        return {'wait': wait, 'throughput': throughput, 'util': util,
                'patience': patience, 'score': score}

# ---------------- INTER-LAYER MUTUAL INFORMATION ----------------
class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.max_input = 2*max_inner_dim
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim if inter_dim is not None else max_inner_dim
        self.weights = {(i,j): np.random.uniform(-0.6,0.6,(self.inter_dim,self.max_input))
                        for i in range(D_graph) for j in range(D_graph) if i!=j}
        self.bias = {(i,j): np.random.uniform(-0.3,0.3,self.inter_dim)
                     for i in range(D_graph) for j in range(D_graph) if i!=j}

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0,self.max_input-len(concat))))[:self.max_input]
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]
        return 1/(1+np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i==j: continue
                if abs(Gmat[i,j])>self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i,j,nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts)<2: return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1,keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2,axis=1)/(A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds,stds)+1e-12)
        np.fill_diagonal(corr,0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat,nested_reps)
        if not acts: return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

# ---------------- UNIFIED ACOR MULTIPLEX ----------------
class GDFCM:
    def __init__(self, candidate_dims, D_graph, inner_archive_size, inner_offspring,
                 outer_archive_size, outer_offspring, synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.inner_archive_size = inner_archive_size
        self.inner_offspring = inner_offspring
        self.outer_archive_size = outer_archive_size
        self.outer_offspring = outer_offspring
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag

        self.nested_reps = [np.zeros(max(candidate_dims)) for _ in range(D_graph)]
        self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []

    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
              steps=100, lr_x=0.001, lr_y=0.001, lr_W=0.001,
              decorrelate_metrics=True):

      # --- Initialize activations ---
      x = target.copy()
      y = target.copy()

      W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
      np.fill_diagonal(W, 0)

      self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx] - target))

      # --- Inner FCM updates ---
      for _ in range(steps):
          z = W.dot(x)
          Theta_grad_z = 2 * z - target

          Theta_grad_x = Theta_grad_z @ W + (y + 1)
          Theta_grad_y = x + 1
          Theta_grad_W = np.outer(Theta_grad_z, x)

          x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
          y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)

          x = np.clip(x, 0, 1)
          y = np.clip(y, 0, 1)

          W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)
          np.fill_diagonal(W, 0)
          W = np.clip(W, -1, 1)

      self.nested_reps[node_idx] = x
      self.l2_after.append(np.linalg.norm(x - target))

      # --- Metric decoupling ---
      metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

      if decorrelate_metrics:
          # Neutralized input
          neutral_y = np.full_like(x, 0.5)
          metrics = metrics_evaluator.compute_node_metrics(node_idx, y=neutral_y)

          # Optional: orthogonalize against inter-layer mean
          inter_tensor = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
          if inter_tensor:
              D = self.D_graph
              inter_mat = np.zeros((D, D))
              for (i,j), vec in inter_tensor.items():
                  inter_mat[i,j] = vec.mean()

              node_vec = np.array([metrics[k] for k in ['wait','throughput','util','patience']])
              for i in range(D):
                  f_vec = inter_mat[i,:]
                  proj = np.dot(node_vec, f_vec) / (np.dot(f_vec,f_vec)+1e-12) * f_vec
                  node_vec -= proj

              metrics['wait'], metrics['throughput'], metrics['util'], metrics['patience'] = node_vec

          # --- Fully decorrelated score ---
          metrics['score'] = metrics['wait'] + metrics['throughput'] + metrics['util'] + metrics['patience']

      else:
          # Compute normally if no decorrelation
          metrics = metrics_evaluator.compute_node_metrics(node_idx, y=x)

      # --- Compute MI score for inter-layer ---
      mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

      return x, y, W, mi_score, metrics

    # ---------- OUTER LOOP ----------
    def run_outer(self, outer_cost_limit=1000):
      metrics_evaluator = MetricsEvaluator(DATA_MATRIX)
      node_metrics_list = []
      raw_scores = []

      # --- Compute node metrics per node ---
      for i, y in enumerate(self.nested_reps):
          metrics = metrics_evaluator.compute_node_metrics(i, y=y)
          node_metrics_list.append(metrics)
          raw_scores.append(metrics['score'])

      raw_scores = np.array(raw_scores)
      total_raw = raw_scores.sum()

      # --- Apply cap to raw metrics ---
      capped_total_raw = total_raw
      if total_raw > outer_cost_limit:
          scale_factor = outer_cost_limit / total_raw
          for metrics in node_metrics_list:
              for key in ['wait', 'throughput', 'util', 'patience', 'score']:
                  metrics[key] *= scale_factor
          raw_scores *= scale_factor
          capped_total_raw = outer_cost_limit

      # --- Compute Fuzzy Metric Tensor contribution ---
      fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
      D = self.D_graph

      # Only consider off-diagonal entries for inter-node interactions
      off_diag_mask = np.ones((D, D), dtype=bool)
      np.fill_diagonal(off_diag_mask, 0)
      fuzzy_score_offdiag = fuzzy_tensor[off_diag_mask].sum()

      # --- Compute per-node contribution ---
      node_contributions = np.zeros(D)
      for i in range(D):
          # Contribution from own metrics
          own_score = raw_scores[i]

          # Contribution from FMT interactions (row i -> others)
          fmt_contrib = fuzzy_tensor[i, :, :].sum() - fuzzy_tensor[i, i, :].sum()  # exclude self
          node_contributions[i] = own_score + self.inter_layer.gamma * fmt_contrib

      # --- Strong decoupling: correlation penalty ---
      inter_tensor = self.print_interactions(return_tensor=True, verbose=False)
      if inter_tensor is None or inter_tensor.shape[2] == 0:
          inter_mean = np.zeros((D, D))
      else:
          inter_mean = inter_tensor.mean(axis=2)  # shape (D,D)

      fmt_mean = fuzzy_tensor.mean(axis=2)  # shape (D,D)

      corr_penalty = 0.0
      for i in range(D):
          fmt_vec = fmt_mean[i, :]
          inter_vec = inter_mean[i, :]
          if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
              corr = np.corrcoef(fmt_vec, inter_vec)[0, 1]
              corr_penalty += abs(corr) ** 2

      corr_penalty /= D
      combined_score = node_contributions.sum() - corr_penalty * 500  # strong decorrelation

      # --- Store for plotting / further analysis ---
      self.capped_node_metrics = node_metrics_list
      self.node_score_contributions = node_contributions
      self.correlation_penalty = corr_penalty

      return node_metrics_list, combined_score, node_contributions





    # ---------- FULL RUN ----------
    def run(self, outer_generations=outer_generations):
      final_metrics = None

      for gen in range(outer_generations):
          mi_scores = []
          node_metrics_list = []

          # --- Inner loop per node ---
          for node_idx in range(self.D_graph):
              dim = self.best_dim_per_node[node_idx]
              target = self.synthetic_targets[node_idx][dim]

              _, _, _, mi_score, metrics = self.run_inner(node_idx, target, dim)
              mi_scores.append(mi_score)
              node_metrics_list.append(metrics)

          # --- Outer loop uses decorrelated metrics ---
          self.capped_node_metrics = node_metrics_list
          _, capped_score, node_contributions = self.run_outer()  # uses self.capped_node_metrics

          print(f"\n--- Generation {gen} Metrics ---")
          for i, m in enumerate(node_metrics_list):
              print(f"Node {i} | " + " | ".join([f"{k}: {v:.2f}" for k,v in m.items()]))

          print(f"\n--- Generation {gen} Node Contributions ---")
          for i, c in enumerate(node_contributions):
              print(f"Node {i}: Contribution = {c:.4f}")

          print(f"Outer Score (capped): {capped_score:.3f}")

          final_metrics = node_metrics_list

      return final_metrics



    # ---------- VISUALIZATIONS ----------
    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14,3))
        for i in range(self.D_graph):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base
            y_true = self.synthetic_targets[i][self.best_dim_per_node[i]]
            if len(y_true)<len(y_sel):
                y_true = np.pad(y_true,(0,len(y_sel)-len(y_true)),"constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1,self.D_graph,i+1)
            plt.fill_between(range(len(y_min)),y_min,y_max,color='skyblue',alpha=0.4,label='Elite Interval')
            plt.plot(y_sel,'k-',lw=2,label='Estimated')
            plt.plot(y_true,'r--',lw=2,label='True')
            plt.ylim(0,1.05)
            plt.title(f"Node {i+1}")
            if i==0: plt.legend()
        plt.tight_layout()
        plt.show()

    def plot_nested_activations(self):
        plt.figure(figsize=(12,3))
        for i,rep in enumerate(self.nested_reps):
            plt.subplot(1,self.D_graph,i+1)
            plt.bar(range(len(rep)),rep,color=plt.cm.plasma(rep))
            plt.ylim(0,1)
            plt.title(f"Node {i+1}")
        plt.tight_layout()
        plt.show()

    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i!=j and abs(self.chosen_Gmat[i,j])>0.02:
                    G.add_edge(i,j,weight=self.chosen_Gmat[i,j])
        node_sizes = [self.best_dim_per_node[i]*200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight']>0 else 'red' for _,_,d in G.edges(data=True)]
        edge_widths = [abs(d['weight'])*3 for _,_,d in G.edges(data=True)]
        pos = nx.circular_layout(G)
        plt.figure(figsize=(6,6))
        nx.draw(G,pos,node_size=node_sizes,node_color='skyblue',
                edge_color=edge_colors,width=edge_widths,arrows=True,with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()
# ---------------- INTERACTIONS INSPECTOR ----------------

    def print_interactions(self, return_tensor=True, verbose=True):
            D_graph = self.D_graph
            inter_dim = self.inter_layer.inter_dim
            inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

            acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
            if not acts:
                if verbose:
                    print("No active edges above threshold.")
                return inter_tensor if return_tensor else None

            for (i, j), vec in acts.items():
                inter_tensor[i, j, :] = vec
                if verbose:
                    act_str = ", ".join([f"{v:.3f}" for v in vec])
                    print(f"Node {i} -> Node {j}: [{act_str}]")
            return inter_tensor if return_tensor else None

        # Move these outside of print_interactions (class-level)
    def print_l2_summary(self):
            print("\nL2 Distances to Target per Node:")
            for idx, (before, after) in enumerate(zip(self.l2_before, self.l2_after)):
                print(f"Node {idx}: Before={before:.4f}, After={after:.4f}")

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i == j:
                        tensor[i, j, :] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i, j]), 0, 1)
                        tensor[i, j, :] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            """
            Computes a Fuzzy Metric Tensor (D_graph x D_graph x num_metrics)
            using current nested reps and node metrics.
            Each slice [i,j,:] represents metrics of node j (optionally weighted by Gmat[i,j])
            """
            metrics_keys = ['wait', 'throughput', 'util', 'patience']
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i==j:
                        tensor[i,j,:] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                        tensor[i,j,:] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor
    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot a heatmap panel for each metric in the FMT.
        Rows: source node i
        Columns: target node j
        """
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:,:,k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j,i,f"{data[i,j]:.2f}",ha='center',va='center',color='white',fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        """
        Computes FMT bounds using pointwise min/max across elite solutions.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        metrics_keys = ['wait','throughput','util','patience']
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D,D,num_metrics,2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        for i in range(D):
            # Generate top_k perturbations around current nested_rep (like in plot_pointwise_minmax_elite)
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)

            # Compute node metrics for each perturbed solution
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx,:] = [m[k] for k in metrics_keys]

            # Compute pointwise min/max across elite solutions
            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            # Fill bounds tensor for all source nodes (i->j)
            for j in range(D):
                tensor_bounds[i,j,:,0] = lower_i
                tensor_bounds[i,j,:,1] = upper_i

        return tensor_bounds


    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        D = self.D_graph
        metrics_keys = ['wait','throughput','util','patience']
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            lower = fmt_tensor_bounds[:,:,k,0]
            upper = fmt_tensor_bounds[:,:,k,1]
            mean_vals = (lower+upper)/2
            range_vals = upper-lower
            max_range = range_vals.max() if range_vals.max()>0 else 1.0
            alphas = 0.2 + 0.8 * range_vals/max_range

            im = axes[k].imshow(mean_vals, cmap='viridis', vmin=0, vmax=mean_vals.max())
            for i in range(D):
                for j in range(D):
                    alpha_val = np.clip(1-alphas[i,j],0,1)
                    rect = plt.Rectangle((j-0.5,i-0.5),1,1,color='white',alpha=alpha_val)
                    axes[k].add_patch(rect)
                    axes[k].text(j,i,f"{lower[i,j]:.1f}\n{upper[i,j]:.1f}",ha='center',va='center',fontsize=8)
            axes[k].set_title(f'FMT Bounds - {key}')
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Mean Metric Value')
        plt.tight_layout()
        plt.show()
    def plot_node_score_contribution(self, metrics_keys=['wait','throughput','util','patience']):
        """
        Plot per-node total score contribution in the SAME STYLE as the FMT plots:
            - uses imshow
            - one panel for: raw, FMT, and total stacked
            - diagonal masked
            - annotated cells
            - node contribution highlighted like your FMT code
        """
        # ---------------------------------------------------------------------
        # 1. Collect node contributions from run_outer()
        # ---------------------------------------------------------------------
        _, _, node_contributions = self.run_outer()
        node_contributions = np.array(node_contributions)
        D = len(node_contributions)

        # ---------------------------------------------------------------------
        # 2. Recompute FMT influence (same style as your FMT plots)
        # ---------------------------------------------------------------------
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)
        total_tensor = fuzzy_tensor.sum(axis=2)           # sum over metrics
        fmt_tensor = total_tensor.copy()
        np.fill_diagonal(fmt_tensor, 0)                   # mask diagonal

        fmt_per_node = fmt_tensor.sum(axis=1)             # row sum
        raw_per_node = node_contributions - fmt_per_node  # everything else

        # Construct matrices for plotting (D×D)
        raw_matrix = np.zeros((D, D))
        fmt_matrix = fmt_tensor
        total_matrix = raw_matrix + fmt_matrix            # raw only on diagonal? no → distribute raw as row diag
        np.fill_diagonal(raw_matrix, raw_per_node)
        total_matrix = raw_matrix + fmt_matrix

        # ---------------------------------------------------------------------
        # 3. Plot - 3 subplots in SAME STYLE as FMT panels
        # ---------------------------------------------------------------------
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))

        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "FMT Interaction Contribution", "Total Contribution"]

        for ax, mat, title in zip(axes, matrices, titles):

            im = ax.imshow(mat, cmap='viridis', vmin=np.min(mat), vmax=np.max(mat))

            # annotate values
            for i in range(D):
                for j in range(D):
                    val = mat[i, j]
                    ax.text(j, i, f"{val:.2f}", ha='center',
                            va='center', color='white', fontsize=8)

            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i+1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i+1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04,
                    label='Contribution Value')

        plt.tight_layout()
        plt.show()

    def correlate_fmt_interactions_per_node(self, fmt_bounds=None, interaction_tensor=None, verbose=True):
        """
        Correlate the FMT bounds with inter-layer interactions per node and per metric.
        Returns a dict of shape: {node_idx: {metric: {'r':..., 'p':...}}}.
        """
        from scipy.stats import pearsonr
        import matplotlib.pyplot as plt

        metrics_keys = ['wait','throughput','util','patience']
        D = self.D_graph

        # Compute tensors if not provided
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=21)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        # Reduce interaction tensor along inter_dim
        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)

        node_correlations = {}

        for i in range(D):
            node_correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT bounds for target node j from source i (mean of lower/upper)
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)  # shape (D,)
                # Interaction tensor for edges from node i to j
                inter_vec = inter_mean[i,:]  # shape (D,)
                # Pearson correlation
                corr, pval = pearsonr(fmt_mean, inter_vec)
                node_correlations[i][key] = {'r': corr, 'p': pval}

                if verbose:
                    print(f"Node {i} | {key}: r = {corr:.3f}, p = {pval:.3e}")
                    plt.figure(figsize=(4,3))
                    plt.scatter(fmt_mean, inter_vec, alpha=0.7, edgecolor='k', color='skyblue')
                    plt.xlabel(f"FMT {key} (Node {i} -> others)")
                    plt.ylabel(f"Interaction mean (Node {i} -> others)")
                    plt.title(f"Node {i} | {key} correlation: r={corr:.3f}")
                    plt.grid(True)
                    plt.show()

        return node_correlations
    def correlation_penalty(self, fmt_bounds=None, interaction_tensor=None):
        """
        Computes a penalty term that is high if per-node FMT metrics correlate with interactions.
        Returns total penalty to subtract from the outer score.
        """
        from scipy.stats import pearsonr

        D = self.D_graph
        metrics_keys = ['wait','throughput','util','patience']

        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=top_k)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)
        total_penalty = 0.0

        for i in range(D):
            for k in range(len(metrics_keys)):
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)
                inter_vec = inter_mean[i,:]
                if np.std(fmt_mean) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr, _ = pearsonr(fmt_mean, inter_vec)
                    total_penalty += abs(corr)  # penalize high correlation

        # normalize by number of nodes × metrics
        total_penalty /= (D * len(metrics_keys))
        return total_penalty


# ---------------- USAGE ----------------
if __name__ == "__main__":
    optimizer = GDFCM(
        candidate_dims, D_graph,
        inner_archive_size, inner_offspring,
        outer_archive_size, outer_offspring,
        synthetic_targets,
        inner_learning, gamma_interlayer=gamma_interlayer,
        causal_flag=False
    )
    metrics_list = optimizer.run()
    optimizer.plot_pointwise_minmax_elite()
    optimizer.plot_nested_activations()
    # Compute FMT with elite bounds
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k)

# Plot as heatmaps
    optimizer.plot_fmt_with_bounds(fmt_elite_bounds)

    # Compute fuzzy multiplex tensor
    fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=True)
    optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

    # Compute FMT with bounds (minimax elite intervals)
    optimizer.plot_node_score_contribution()
    optimizer.plot_outer_fuzzy_graph()
  #  optimizer.print_interactions()
    tensor = optimizer.print_interactions()

    print("Tensor shape:", tensor.shape,'\n',tensor)
    # Compute tensors first
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k)
    interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

    # Get per-node, per-metric correlations
    node_metric_corrs = optimizer.correlate_fmt_interactions_per_node(
        fmt_bounds=fmt_elite_bounds,
        interaction_tensor=interaction_tensor
    )
    import networkx as nx
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
D_graph = len(optimizer.nested_reps)
tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

# ---------------- Outer nodes (hubs) ----------------
G_outer = nx.DiGraph()
for i in range(D_graph):
    G_outer.add_node(i)
for i in range(D_graph):
    for j in range(D_graph):
        if i != j and np.any(tensor[i,j,:] != 0):
            # Shift to signed weights: 0.5 -> 0, <0.5 negative, >0.5 positive
            mean_weight = 2 * (np.mean(tensor[i,j,:]) - 0.5)
            G_outer.add_edge(i, j, weight=mean_weight)

# Outer spring layout
pos_outer_2d = nx.circular_layout(G_outer, scale=5)
pos_outer = np.array([[x, y, 0] for x, y in pos_outer_2d.values()])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

# Plot outer nodes
for i in range(D_graph):
    ax.scatter(*pos_outer[i], s=300, color='skyblue')
    ax.text(*pos_outer[i], f'Node {i}', color='black')

# Plot outer edges with positive/negative colors

for i, j, data in G_outer.edges(data=True):
    x_vals = [pos_outer[i,0], pos_outer[j,0]]
    y_vals = [pos_outer[i,1], pos_outer[j,1]]
    z_vals = [pos_outer[i,2], pos_outer[j,2]]

    # Positive = bright green, Negative = bright red
    color = 'green' if data['weight'] > 0 else 'red'
    linewidth = 2 + 4*abs(data['weight'])  # scale width by magnitude
    ax.plot(x_vals, y_vals, z_vals, color=color, linewidth=linewidth)
# ---------------- Inner FCMs (small circular around hub) ----------------
for i, rep in enumerate(optimizer.nested_reps):
    dims = len(rep)
    angle = np.linspace(0, 2*np.pi, dims, endpoint=False)
    radius = 0.8  # small circle
    xs = pos_outer[i,0] + radius * np.cos(angle)
    ys = pos_outer[i,1] + radius * np.sin(angle)
    zs = pos_outer[i,2] + rep  # activation as height

    # Plot inner nodes
    ax.scatter(xs, ys, zs, c=rep, cmap='plasma', s=50)

    # Connect inner nodes in circle
    for k in range(dims):
        ax.plot([xs[k], xs[(k+1)%dims]], [ys[k], ys[(k+1)%dims]], [zs[k], zs[(k+1)%dims]], color='gray', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Activation')
ax.set_title('Outer Nodes with Inner FCMs (Signed correlations)')
plt.show()




In [ ]:
import numpy as np

class GDFCMPredictorForward:
    """
    Forward predictor for GDFCM.
    Can handle new input data (DATA_MATRIX) and compute:
        - Node activations
        - Node metrics
        - Inter-layer MI scores
        - Total contributions and outer score
    """
    def __init__(self, trained_gdfcm: GDFCM):
        self.gdfcm = trained_gdfcm
        self.D_graph = trained_gdfcm.D_graph
        self.nested_reps = trained_gdfcm.nested_reps
        self.chosen_Gmat = trained_gdfcm.chosen_Gmat
        self.inter_layer = trained_gdfcm.inter_layer

    def predict_node(self, node_idx, data_row):
        """
        Predict metrics and activations for a single node given a new data row.
        """
        # Use stored nested_rep as starting point
        x = self.nested_reps[node_idx].copy()
        # Optional: you could combine with data_row to modify x
        # For now, we just compute metrics using data_row as the input
        metrics_evaluator = MetricsEvaluator(np.array([data_row]))
        metrics = metrics_evaluator.compute_node_metrics(0, y=x)

        # Inter-layer MI
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return {'activations': x, 'metrics': metrics, 'mi_score': mi_score}

    def predict_all_nodes(self, new_data_matrix):
        """
        Predict metrics and activations for all nodes using new data.
        new_data_matrix: shape (D_graph, num_features)
        """
        results = []
        for node_idx in range(self.D_graph):
            data_row = new_data_matrix[node_idx % len(new_data_matrix)]
            node_result = self.predict_node(node_idx, data_row)
            results.append(node_result)
        return results

    def predict_scores(self):
        """
        Compute per-node contributions and total outer score.
        """
        _, total_score, node_contributions = self.gdfcm.run_outer()
        return {'node_contributions': node_contributions, 'total_score': total_score}
# Suppose optimizer is your trained GDFCM
predictor = GDFCMPredictorForward(optimizer)

# New data: same number of nodes, each with same num_features
new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])

# Predict all nodes
predictions = predictor.predict_all_nodes(new_DATA_MATRIX)

for idx, node_pred in enumerate(predictions):
    print(f"Node {idx} Metrics:", node_pred['metrics'])
    print(f"Node {idx} Activations:", node_pred['activations'])
    print(f"Node {idx} MI Score:", node_pred['mi_score'])

# Optional: compute total contributions
score_info = predictor.predict_scores()
print("Node Contributions:", score_info['node_contributions'])
print("Total Score:", score_info['total_score'])



In [ ]:
class GDFCMPredictorAdaptive:
    """
    Adaptive forward predictor for GDFCM.
    - Accepts new data per node.
    - Updates activations slightly using a mini inner-loop.
    - Computes metrics, inter-layer MI, and outer score contributions.
    """
    def __init__(self, trained_gdfcm: GDFCM, lr_x=0.01, lr_steps=10):
        self.gdfcm = trained_gdfcm
        self.D_graph = trained_gdfcm.D_graph
        self.nested_reps = [rep.copy() for rep in trained_gdfcm.nested_reps]
        self.chosen_Gmat = trained_gdfcm.chosen_Gmat
        self.inter_layer = trained_gdfcm.inter_layer
        self.lr_x = lr_x
        self.lr_steps = lr_steps

    def adapt_node(self, node_idx, data_row):
        """
        Mini inner-loop: slightly update activations based on new data row.
        Handles shape mismatch by projecting new data to node activation dim.
        """
        x = self.nested_reps[node_idx].copy()
        dim = len(x)

        # Simple linear projection of new data to activation space
        if len(data_row) != dim:
            # Use mean pooling to reduce or slice if too large
            if len(data_row) > dim:
                target = data_row[:dim]
            else:
                # Pad with 0.5
                target = np.pad(data_row, (0, dim - len(data_row)), 'constant', constant_values=0.5)
        else:
            target = data_row

        target = np.clip(target, 0, 1)

        # Mini inner-loop update
        for _ in range(self.lr_steps):
            grad = x - target
            x -= self.lr_x * grad
            x = np.clip(x, 0, 1)

        self.nested_reps[node_idx] = x

        # Compute metrics
        metrics_evaluator = MetricsEvaluator(np.array([data_row]))
        metrics = metrics_evaluator.compute_node_metrics(0, y=x)

        # Inter-layer MI
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return {'activations': x, 'metrics': metrics, 'mi_score': mi_score}


    def adapt_all_nodes(self, new_data_matrix):
        """
        Update activations for all nodes given new data matrix.
        """
        results = []
        for node_idx in range(self.D_graph):
            data_row = new_data_matrix[node_idx % len(new_data_matrix)]
            node_result = self.adapt_node(node_idx, data_row)
            results.append(node_result)
        return results

    def compute_outer_score(self):
        """
        Compute node contributions and total outer score using current nested_reps.
        """
        _, total_score, node_contributions = self.gdfcm.run_outer()
        return {'node_contributions': node_contributions, 'total_score': total_score}
# Initialize adaptive predictor
adaptive_predictor = GDFCMPredictorAdaptive(optimizer, lr_x=0.02, lr_steps=15)

# New data
new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])

# Adapt activations to new data
predictions = adaptive_predictor.adapt_all_nodes(new_DATA_MATRIX)

for idx, node_pred in enumerate(predictions):
    print(f"Node {idx} Metrics:", node_pred['metrics'])
    print(f"Node {idx} Activations:", node_pred['activations'])
    print(f"Node {idx} MI Score:", node_pred['mi_score'])

# Compute outer node contributions after adaptation
score_info = adaptive_predictor.compute_outer_score()
print("Node Contributions:", score_info['node_contributions'])
print("Total Score:", score_info['total_score'])


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error

pred_metrics = []
true_metrics = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    pred = adaptive_predictor.adapt_node(idx, data_row)
    pred_metrics.append([pred['metrics'][k] for k in ['wait','throughput','util','patience']])
    true_metrics.append([MetricsEvaluator(new_DATA_MATRIX).compute_node_metrics(idx)[k]
                         for k in ['wait','throughput','util','patience']])

pred_metrics = np.array(pred_metrics)
true_metrics = np.array(true_metrics)


def evaluate_predictions(true_metrics, pred_metrics, metric_names=None):
    """
    Evaluate multi-metric predictions per node and overall.

    Args:
        true_metrics: np.array, shape (num_nodes, num_metrics)
        pred_metrics: np.array, same shape as true_metrics
        metric_names: list of metric names (optional)

    Returns:
        dict with evaluation metrics
    """
    true_metrics = np.array(true_metrics)
    pred_metrics = np.array(pred_metrics)

    if metric_names is None:
        metric_names = [f'Metric {i}' for i in range(true_metrics.shape[1])]

    num_nodes, num_metrics = true_metrics.shape

    results = {'per_metric': {}, 'overall': {}}

    # Per metric
    for i, name in enumerate(metric_names):
        t = true_metrics[:, i]
        p = pred_metrics[:, i]
        mae = mean_absolute_error(t, p)
        rmse = np.sqrt(mean_squared_error(t, p))
        r2 = r2_score(t, p)
        corr = np.corrcoef(t, p)[0,1]
        mape = np.mean(np.abs((t - p) / (t + 1e-12))) * 100
        results['per_metric'][name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'Pearson': corr,
            'MAPE (%)': mape
        }

    # Overall
    mae_overall = mean_absolute_error(true_metrics.flatten(), pred_metrics.flatten())
    rmse_overall = np.sqrt(mean_squared_error(true_metrics.flatten(), pred_metrics.flatten()))
    r2_overall = r2_score(true_metrics.flatten(), pred_metrics.flatten())

    # Cosine similarity (averaged over nodes)
    cos_sim = np.mean([cosine_similarity(t.reshape(1,-1), p.reshape(1,-1))[0,0]
                       for t,p in zip(true_metrics, pred_metrics)])

    results['overall'] = {
        'MAE': mae_overall,
        'RMSE': rmse_overall,
        'R2': r2_overall,
        'CosineSim': cos_sim
    }

    return results
# Suppose true_metrics and pred_metrics have shape (D_graph, 4)
metrics_eval = evaluate_predictions(true_metrics, pred_metrics, metric_names=['wait','throughput','util','patience'])

# Print per-metric evaluation
for metric, vals in metrics_eval['per_metric'].items():
    print(f"{metric}: {vals}")

# Print overall evaluation
print("\nOverall metrics:", metrics_eval['overall'])



In [ ]:
import numpy as np

class PureFCMPredictor:
    def __init__(self, D_graph, dims_per_node, lr_x=0.01, steps=200, seed=42):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.dims_per_node = dims_per_node
        self.lr_x = lr_x
        self.steps = steps
        # Initialize per-node weights
        self.Ws = [np.random.uniform(-0.6, 0.6, (d, d)) for d in dims_per_node]
        for W in self.Ws:
            np.fill_diagonal(W, 0)
        # Initialize node states
        self.xs = [np.zeros(d) for d in dims_per_node]

    def fit_node(self, node_idx, target):
        x = self.xs[node_idx].copy()
        W = self.Ws[node_idx]

        for _ in range(self.steps):
            x = x + self.lr_x * (target - x)  # simple L2 update
            x = np.clip(x, 0, 1)

        self.xs[node_idx] = x
        return x

    def predict(self, data_matrix):
        preds = []
        for i in range(self.D_graph):
            # Here you could use the mean of row features as a simple target proxy
            target = data_matrix[i % len(data_matrix), :self.dims_per_node[i]]
            pred = self.fit_node(i, target)
            preds.append(pred)
        return preds
# Suppose DATA_MATRIX is your node feature matrix
dims_per_node = [len(rep) for rep in optimizer.nested_reps]  # same as before
pure_fcm = PureFCMPredictor(D_graph=len(dims_per_node), dims_per_node=dims_per_node)

predictions = pure_fcm.predict(DATA_MATRIX)

# Convert predictions to metrics
metrics_evaluator = MetricsEvaluator(DATA_MATRIX)
pred_metrics = [metrics_evaluator.compute_node_metrics(i, y=pred) for i, pred in enumerate(predictions)]


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error

pred_metrics = []
true_metrics = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    pred = adaptive_predictor.adapt_node(idx, data_row)
    pred_metrics.append([pred['metrics'][k] for k in ['wait','throughput','util','patience']])
    true_metrics.append([MetricsEvaluator(new_DATA_MATRIX).compute_node_metrics(idx)[k]
                         for k in ['wait','throughput','util','patience']])

pred_metrics = np.array(pred_metrics)
true_metrics = np.array(true_metrics)


def evaluate_predictions(true_metrics, pred_metrics, metric_names=None):
    """
    Evaluate multi-metric predictions per node and overall.

    Args:
        true_metrics: np.array, shape (num_nodes, num_metrics)
        pred_metrics: np.array, same shape as true_metrics
        metric_names: list of metric names (optional)

    Returns:
        dict with evaluation metrics
    """
    true_metrics = np.array(true_metrics)
    pred_metrics = np.array(pred_metrics)

    if metric_names is None:
        metric_names = [f'Metric {i}' for i in range(true_metrics.shape[1])]

    num_nodes, num_metrics = true_metrics.shape

    results = {'per_metric': {}, 'overall': {}}

    # Per metric
    for i, name in enumerate(metric_names):
        t = true_metrics[:, i]
        p = pred_metrics[:, i]
        mae = mean_absolute_error(t, p)
        rmse = np.sqrt(mean_squared_error(t, p))
        r2 = r2_score(t, p)
        corr = np.corrcoef(t, p)[0,1]
        mape = np.mean(np.abs((t - p) / (t + 1e-12))) * 100
        results['per_metric'][name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'Pearson': corr,
            'MAPE (%)': mape
        }

    # Overall
    mae_overall = mean_absolute_error(true_metrics.flatten(), pred_metrics.flatten())
    rmse_overall = np.sqrt(mean_squared_error(true_metrics.flatten(), pred_metrics.flatten()))
    r2_overall = r2_score(true_metrics.flatten(), pred_metrics.flatten())

    # Cosine similarity (averaged over nodes)
    cos_sim = np.mean([cosine_similarity(t.reshape(1,-1), p.reshape(1,-1))[0,0]
                       for t,p in zip(true_metrics, pred_metrics)])

    results['overall'] = {
        'MAE': mae_overall,
        'RMSE': rmse_overall,
        'R2': r2_overall,
        'CosineSim': cos_sim
    }

    return results
# Suppose true_metrics and pred_metrics have shape (D_graph, 4)
metrics_eval = evaluate_predictions(true_metrics, pred_metrics, metric_names=['wait','throughput','util','patience'])

# Print per-metric evaluation
for metric, vals in metrics_eval['per_metric'].items():
    print(f"{metric}: {vals}")

# Print overall evaluation
print("\nOverall metrics:", metrics_eval['overall'])



In [ ]:
# ---------------- PIPELINE ----------------
# ---------------- Pure FCM Predictor ----------------
class PureFCMPredictor:
    def __init__(self, D_graph, candidate_dims, synthetic_targets, lr_x=0.01, steps=50):
        self.D_graph = D_graph
        self.candidate_dims = candidate_dims
        self.synthetic_targets = synthetic_targets
        self.lr_x = lr_x
        self.steps = steps
        self.nested_reps = [np.zeros(max(candidate_dims)) for _ in range(D_graph)]

    def adapt_node(self, node_idx, data_row):
        target = self.synthetic_targets[node_idx][self.candidate_dims[node_idx]]
        x = target.copy()
        # simple gradient descent to adapt x
        for _ in range(self.steps):
            grad = x - target  # L2 loss gradient
            x -= self.lr_x * grad
            x = np.clip(x, 0, 1)
        self.nested_reps[node_idx] = x
        # compute metrics
        metrics = MetricsEvaluator(data_row.reshape(1,-1)).compute_node_metrics(0, y=x)
        return {'rep': x, 'metrics': metrics}

# Instantiate the Pure FCM predictor
fcm_predictor = PureFCMPredictor(D_graph=D_graph,
                                 candidate_dims=candidate_dims,
                                 synthetic_targets=synthetic_targets)

import numpy as np

# 1. Generate predictions for Multiplex model
pred_metrics_multiplex = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    pred = adaptive_predictor.adapt_node(idx, data_row)
    pred_metrics_multiplex.append([pred['metrics'][k] for k in ['wait','throughput','util','patience']])
pred_metrics_multiplex = np.array(pred_metrics_multiplex)

# 2. Generate predictions for Pure FCM model
pred_metrics_fcm = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    pred = fcm_predictor.adapt_node(idx, data_row)  # fcm_predictor: Pure FCM predictor
    pred_metrics_fcm.append([pred['metrics'][k] for k in ['wait','throughput','util','patience']])
pred_metrics_fcm = np.array(pred_metrics_fcm)

# 3. Compute true metrics
true_metrics = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    true_metrics.append([MetricsEvaluator(new_DATA_MATRIX).compute_node_metrics(idx)[k]
                         for k in ['wait','throughput','util','patience']])
true_metrics = np.array(true_metrics)

# 4. Evaluate predictions
def evaluate_predictions(true_metrics, pred_metrics, metric_names=None):
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.metrics.pairwise import cosine_similarity

    true_metrics = np.array(true_metrics)
    pred_metrics = np.array(pred_metrics)
    if metric_names is None:
        metric_names = [f'Metric {i}' for i in range(true_metrics.shape[1])]

    results = {'per_metric': {}, 'overall': {}}
    for i, name in enumerate(metric_names):
        t, p = true_metrics[:,i], pred_metrics[:,i]
        mae = mean_absolute_error(t,p)
        rmse = np.sqrt(mean_squared_error(t,p))
        r2 = r2_score(t,p)
        corr = np.corrcoef(t,p)[0,1]
        mape = np.mean(np.abs((t-p)/(t+1e-12))) * 100
        results['per_metric'][name] = {'MAE': mae,'RMSE': rmse,'R2': r2,'Pearson': corr,'MAPE (%)': mape}

    mae_overall = mean_absolute_error(true_metrics.flatten(), pred_metrics.flatten())
    rmse_overall = np.sqrt(mean_squared_error(true_metrics.flatten(), pred_metrics.flatten()))
    r2_overall = r2_score(true_metrics.flatten(), pred_metrics.flatten())
    cos_sim = np.mean([cosine_similarity(t.reshape(1,-1), p.reshape(1,-1))[0,0]
                       for t,p in zip(true_metrics, pred_metrics)])
    results['overall'] = {'MAE': mae_overall,'RMSE': rmse_overall,'R2': r2_overall,'CosineSim': cos_sim}
    return results

metrics_multiplex = evaluate_predictions(true_metrics, pred_metrics_multiplex,
                                          metric_names=['wait','throughput','util','patience'])
metrics_fcm = evaluate_predictions(true_metrics, pred_metrics_fcm,
                                   metric_names=['wait','throughput','util','patience'])

# 5. Compare models
def compare_models(true_metrics, pred1, pred2, metric_names=None):
    pred1 = np.array(pred1)
    pred2 = np.array(pred2)
    true_metrics = np.array(true_metrics)
    diff = np.abs(true_metrics - pred2) - np.abs(true_metrics - pred1)  # positive -> model1 better
    return diff, diff.mean(axis=0), diff.mean(axis=1)

diff_matrix, avg_per_metric, avg_per_node = compare_models(true_metrics,
                                                           pred_metrics_multiplex,
                                                           pred_metrics_fcm,
                                                           metric_names=['wait','throughput','util','patience'])

# 6. Print summaries
print("Multiplex Overall:", metrics_multiplex['overall'])
print("Pure FCM Overall:", metrics_fcm['overall'])
print("Average improvement per metric:", avg_per_metric)
print("Average improvement per node:", avg_per_node)
